In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
param_date = '20240715'

In [3]:
import os

list_file = sorted(list(os.walk(f'./{param_date}/9.type'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

list_file

['make_1.ABARTH.xlsx',
 'make_10.AMERICAN AUSTIN.xlsx',
 'make_100.GOLIATH.xlsx',
 'make_101.GORDON-KEEBLE.xlsx',
 'make_102.GRAHAM.xlsx',
 'make_103.GRAHAM-PAIGE.xlsx',
 'make_104.GRIFFITH.xlsx',
 'make_105.HEALEY.xlsx',
 'make_106.HENRY J.xlsx',
 'make_107.HILLMAN.xlsx',
 'make_108.HINO.xlsx',
 'make_109.HISPANO-SUIZA.xlsx',
 'make_11.AMERICAN BANTAM.xlsx',
 'make_110.HONDA-serial_1.xlsx',
 'make_110.HONDA-serial_2.xlsx',
 'make_110.HONDA-serial_3.xlsx',
 'make_110.HONDA-serial_4.xlsx',
 'make_110.HONDA-serial_5.xlsx',
 'make_110.HONDA-serial_6.xlsx',
 'make_110.HONDA-serial_7.xlsx',
 'make_110.HONDA-serial_8.xlsx',
 'make_110.HONDA-serial_9.xlsx',
 'make_111.HOTCHKISS.xlsx',
 'make_112.HRG.xlsx',
 'make_113.HUDSON.xlsx',
 'make_114.HUMBER.xlsx',
 'make_115.HUMMER-serial_1.xlsx',
 'make_115.HUMMER-serial_2.xlsx',
 'make_116.HUPMOBILE.xlsx',
 'make_117.HYUNDAI-serial_1.xlsx',
 'make_117.HYUNDAI-serial_2.xlsx',
 'make_117.HYUNDAI-serial_3.xlsx',
 'make_117.HYUNDAI-serial_4.xlsx',
 'mak

In [ ]:
for file in list_file[list_file.index(''):]:

    # = = = = = = = = = = = = = = =

    import pandas as pd

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./{param_date}/9.type/{file}',
                           header=0,
                           dtype=str).fillna('')

    input_['Make No'] = input_['Make No'].astype(int)
    input_['Model No'] = input_['Model No'].astype(int)
    input_['Engine No'] = input_['Engine No'].astype(int)
    input_['Year'] = input_['Year'].astype(int)
    input_['Group No'] = input_['Group No'].astype(int)
    input_['Type No'] = input_['Type No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =

    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            import requests
            import json

            import sys
            sys.path.append('../../00.Tools')
            from crawler_configuration import get_header, get_proxy

            payload = {'jsn': {'tab': 'catalog',
                               'make': input_.loc[a, 'Make'],
                               'year': int(input_.loc[a, 'Year']),
                               'model': input_.loc[a, 'Model'],
                               'carcode': input_.loc[a, 'Car Code'],
                               'engine': input_.loc[a, 'Engine'],
                               'groupname': input_.loc[a, 'Group'],
                               'parttype': input_.loc[a, 'Type Code'],
                               'nodetype': 'parttype'}}

            data = {'func': 'navnode_fetch',
                    'payload': json.dumps(payload),
                    'api_json_request': '1'}

            try:
                b = 0
                while True:

                    while True:
                        b += 1
                        try:
                            resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                                 data=data,
                                                 headers=get_header(),
                                                 proxies=get_proxy(),
                                                 timeout=(10, 10))
                            break
                        except:
                            continue

                    if resp.status_code == 200:
                        json_ = resp.json()
                        if 'redirect_to_url' in json_:
                            continue
                        elif json_['jsn']['tab'] == 'catalog' and json_['jsn']['make'] == input_.loc[a, 'Make'] and json_['jsn']['year'] == input_.loc[a, 'Year'] and json_['jsn']['model'] == input_.loc[a, 'Model'] and json_['jsn']['carcode'] == input_.loc[a, 'Car Code'] and json_['jsn']['engine'] == input_.loc[a, 'Engine'] and json_['jsn']['groupname'] == input_.loc[a, 'Group'] and json_['jsn']['parttype'] == input_.loc[a, 'Type Code'] and json_['jsn']['nodetype'] == 'parttype' and 'collected_javascript' in json_:
                            break

                # = = = = = = = = = = = = = = =

                from bs4 import BeautifulSoup
                from lxml import etree

                soup = BeautifulSoup(json_['html_fill_sections']['navchildren[]'], 'lxml')
                html = etree.HTML(str(soup))

                # = = = = = = = = = = = = = = =

                list_tbody = html.xpath('//tbody[contains(@class, "listing-inner") and (contains(@class, "altrow-a-1") or contains(@class, "altrow-a-0"))]')

                # = = = = = = = = = = = = = = =

                if not list_tbody:
                    raise

                # = = = = = = = = = = = = = = =

                list_list_note = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/span[@class="span-link-underline-remover"]/text()') for tbody in list_tbody]
                list_note = ['\n'.join([note.strip() for note in list_note]) for list_note in list_list_note]

                # = = = = = = = = = = = = = = =

                list_list_info = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row-moreinfo-truck"]/a[@class="ra-btn ra-btn-moreinfo"]/@href') for tbody in list_tbody]
                list_info = [list_info[0].strip() if list_info and list_info[0].startswith('https://www.rockauto.com/en/moreinfo.php?') else '' for list_info in list_list_info]

                # = = = = = = = = = = = = = = =

                list_dict_note = []
                list_span = [tbody.xpath('./tr[1]/td/div[@class="listing-text-row"]/span[contains(@title, "Please make sure")]')[0] for tbody in list_tbody]

                for span in list_span:
                    dict_note = {}
                    list_note_2 = span.xpath('./span[@class="listing-footnote-text"]/text()')
                    list_market = span.xpath('./span[@class="listing-footnote-text"]/a[@class="appflag"]/@data-market')
                    if list_market:
                        dict_note['separate'] = {}
                        for i, note, market in zip(range(len(list_note_2)), list_note_2, list_market):
                            dict_note['separate'][str(i)] = {'market': market.strip(), 'note': note.strip()}
                    else:
                        dict_note = {'combined': {'note': ''.join(list_note_2).strip()}}

                    list_dict_note.append(dict_note)

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td[contains(@class, "listing-inner-content")]/input[contains(@id, "listing_data_essential[__GIP__")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_key = [json_['partkey'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_json = [json.loads(tbody.xpath('./tr[1]/td/input[contains(@id, "listing_data_supplemental[")]/@value')[0]) for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_part_number = [json_['partnumber'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_manufacturer = [json_['catalogname'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_partition = [json_['sortgrouptext'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_parameter = [json_['paramdesc'] for json_ in list_json]

                # = = = = = = = = = = = = = = =

                list_list_oem = [tbody.xpath('./tr[1]/td/span[contains(@title, "Replaces these Alternate")]/text()') for tbody in list_tbody]
                list_oem = [','.join(list_oem) for list_oem in list_list_oem]
                list_oem = [';'.join([oem_split.strip() for oem_split in oem.split(',')]) for oem in list_oem]

                # = = = = = = = = = = = = = = =

                list_list_src = [json.loads(tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value')[0])['Slots'] if tbody.xpath('./tr[1]/td/div/input[contains(@id, "jsninlineimg[")]/@value') else [] for tbody in list_tbody]

                # = = = = = = = = = = = = = = =

                list_dict_src = []
                for list_src in list_list_src:
                    dict_src = {}
                    for i, src in zip(range(len(list_src)), list_src):
                        dict_src[str(i)] = 'https://www.rockauto.com' + src['ImageData']['Full'].strip()

                    list_dict_src.append(dict_src)

                # = = = = = = = = = = = = = = =

                import re

                list_price = [etree.tostring(tbody.xpath('./tr[1]/td[contains(@id, "listingtd[") and contains(@id, "][price]")]/span/span/span[contains(@id, "dprice[") and contains(@id, "][v]")]')[0]).decode('utf-8') for tbody in list_tbody]
                list_dict_price = [{'0': re.sub(r'<.*?>', '', price.strip())} for price in list_price]

                # = = = = = = = = = = = = = = =

                list_list_li = [tbody.xpath('./tr[1]/td/div/div/span/span[contains(@id, "ddrepl[optionchoice[__")]/table/tr/td[1]/div/div/ul/li') for tbody in list_tbody]
                for i, list_li in zip(range(len(list_list_li)), list_list_li):
                    if list_li:
                        list_dict_price[i]['1'] = {}
                        for j, li in zip(range(len(list_li[1:])), list_li[1:]):
                            list_dict_price[i]['1'][str(j)] = re.sub(r' +', ' ', re.sub(r'<.*?>', ' ', etree.tostring(li.xpath('./div[@class="outer"]/div[@class="inner"]')[0]).decode('utf-8'))).strip()

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame({'Make No': input_.loc[a, 'Make No'],
                                        'Make': input_.loc[a, 'Make'],
                                        'Model No': input_.loc[a, 'Model No'],
                                        'Model': input_.loc[a, 'Model'],
                                        'Engine No': input_.loc[a, 'Engine No'],
                                        'Engine': input_.loc[a, 'Engine'],
                                        'Car Code': input_.loc[a, 'Car Code'],
                                        'Year': input_.loc[a, 'Year'],
                                        'Group No': input_.loc[a, 'Group No'],
                                        'Group': input_.loc[a, 'Group'],
                                        'Type No': input_.loc[a, 'Type No'],
                                        'Type': input_.loc[a, 'Type'],
                                        'Type Code': input_.loc[a, 'Type Code'],
                                        'Tab 1 Url': input_.loc[a, 'Tab 1 Url'],
                                        'Part No': [i+1 for i in range(len(list_part_number))],
                                        'Partition': list_partition,
                                        'Manufacturer': list_manufacturer,
                                        'Part Number': list_part_number,
                                        'Key': list_key,
                                        'JOIN_MPNTCK': [f'''{manufacturer};{part_number};{input_.loc[a, 'Type Code']};{key}''' for manufacturer, part_number, key in zip(list_manufacturer, list_part_number, list_key)],
                                        'Tab 1 Note 1': list_note,
                                        'Tab 1 Note 2': [json.dumps(dict_note) for dict_note in list_dict_note],
                                        'Parameter': list_parameter,
                                        'Json_Price': [json.dumps(dict_price) for dict_price in list_dict_price],
                                        'OEM': list_oem,
                                        'Info': list_info,
                                        'Json_Src': [json.dumps(dict_src) for dict_src in list_dict_src]})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([{'Make No': input_.loc[a, 'Make No'],
                                         'Make': input_.loc[a, 'Make'],
                                         'Model No': input_.loc[a, 'Model No'],
                                         'Model': input_.loc[a, 'Model'],
                                         'Engine No': input_.loc[a, 'Engine No'],
                                         'Engine': input_.loc[a, 'Engine'],
                                         'Car Code': input_.loc[a, 'Car Code'],
                                         'Year': input_.loc[a, 'Year'],
                                         'Group No': input_.loc[a, 'Group No'],
                                         'Group': input_.loc[a, 'Group'],
                                         'Type No': input_.loc[a, 'Type No'],
                                         'Type': input_.loc[a, 'Type'],
                                         'Type Code': input_.loc[a, 'Type Code'],
                                         'Tab 1 Url': input_.loc[a, 'Tab 1 Url']}])

                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'Model No']}.{input_.loc[a, 'Model']} > {input_.loc[a, 'Engine No']}.{input_.loc[a, 'Engine']} > {input_.loc[a, 'Year']} > {input_.loc[a, 'Group No']}.{input_.loc[a, 'Group']} > {input_.loc[a, 'Type No']}.{input_.loc[a, 'Type']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(9):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year', 'Group No', 'Type No', 'Part No'],
                                                    ascending=[True, True, True, False, True, True, True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./{param_date}/10.part_1/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['Make No', 'Model No', 'Engine No', 'Year', 'Group No', 'Type No'],
                                                ascending=[True, True, True, False, True, True],
                                                ignore_index=True)
        output_error.to_excel(f'''./{param_date}/10.part_1/0.error/{file.removesuffix('.xlsx')}-part_1_error.xlsx''', index=False)
        print('爬虫存在error')
        print()
    print('Done ~')

[状态：ok，尝试次数：1] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 11.Cylinder Repair Sleeve
[剩余数量：1000] - [当前时间：10:55:16]

[状态：ok，尝试次数：14] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 8.Crankshaft Seal
[剩余数量：999] - [当前时间：10:55:17]

[状态：ok，尝试次数：1] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 12.Engine Block Heater
[剩余数量：998] - [当前时间：10:55:18]

[状态：ok，尝试次数：1] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 13.Engine Kit Gasket Set
[剩余数量：997] - [当前时间：10:55:18]

[状态：ok，尝试次数：1] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 14.Exhaust Valve
[剩余数量：996] - [当前时间：10:55:19]

[状态：ok，尝试次数：1] - 6.63% - make_12.AMERICAN MOTORS-serial_6 > 12.JAVELIN > 8.6.6L 401cid V8 > 1973 > 10.Engine > 15.Expansion Plug
[剩余数量：995] - [当前时间：10:55:20]

[状态：ok，尝试次数：18] - 6.63% -